<a href="https://colab.research.google.com/github/trista-paul/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/DS_Unit_4_Sprint_3_Neural_Nets_Spring_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron - neurons are a unit of data transfer in the neural network. They are like real life neurons inasfar as they have thresholds for which data to let through (defined by activation).

- Input Layer - the first layer of a neural network, shaping the initial input. Takes the parameter input_dim or input_shape.

- Hidden Layer - layers that multiply input by weight and apply activation function.

- Output Layer - final layer of neural network that applies final activation function to get layer in output shape. The neuron number is usually 1.

- Activation - function that reshapes data so it can be accepted as input for the next layer/final output.

- Backpropagation - short for backwards propogation of errors, a second weight update after feed forward

 YOUR ANSWER HERE

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [0]:
import numpy as np

X = [[1, 1, 1],
     [1, 0, 1],
     [0, 1, 1],
     [0, 0, 1]]

y = [[1],
     [0],
     [0],
     [0]]

In [0]:
import pandas as pd
X = pd.DataFrame(X)
y = pd.DataFrame(y)

In [0]:
class FPerceptron(object):
  def __init__(self, rate=0.01, niter=100):
    self.rate = rate
    self.niter = niter
    
  def sigmoid(x):
    return 1 / (1 + np.exp(-x))

  def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))
    
  def fit(self, X, y):
    for i in range(self.niter):
      self.weights = 2 * np.random.random((X.shape[1],1)) - 1
      self.weighted_sum = np.dot(X, self.weights)
      self.activated_output = sigmoid(self.weighted_sum)
      self.error = y - activated_output
      self.adjustments = self.error * sigmoid_derivative(self.activated_output)
      self.weights += np.dot(X.T, self.adjustments)
      return self.activated_output

In [17]:
FF = FPerceptron()
FF.fit(X, y)

array([[0.27956589],
       [0.41201126],
       [0.46799344],
       [0.61366809]])

## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [18]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [19]:
df.shape[1]

14

In [0]:
X = df.drop(columns = 'target').values
y = df['target'].values

In [21]:
X.shape

(303, 13)

In [22]:
y.shape

(303,)

In [0]:
class MultilayerPerceptron(object):
  def __init__(self, rate=0.01, niter=100):
    self.rate = rate
    self.niter = niter
    
  def sigmoid(x):
    return 1 / (1 + np.exp(-x))

  def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))
    
  def forward(self, X, y):
    for i in range(self.niter):
      self.weights = 2 * np.random.random((X.shape[1],1)) - 1
      self.weighted_sum = np.dot(X, self.weights)
      self.activated_output = sigmoid(self.weighted_sum)
      self.error = y - activated_output
      self.adjustments = self.error * sigmoid_derivative(self.activated_output)
      self.weights += np.dot(X.T, self.adjustments)
      return self.activated_output
    
  def backward(self, X, y)

In [26]:
MLP = MultiLayerPerceptron()
MLP.train(X, y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


ValueError: ignored

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [30]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [0]:
X = keras.utils.normalize(X, axis=-1, order=2)

In [32]:
model = Sequential()

#input layer: input_dim is number of features
model.add(Dense(12, input_dim=13, activation='relu'))

#hidden layer 1
model.add(Dense(12, activation='relu'))

#output layer: 1 node and sigmoid
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['mse'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                168       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13        
Total params: 337
Trainable params: 337
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(Xtrain, ytrain, batch_size=128, epochs=50, verbose=0, validation_split=0.33)